# Systèmes DAE en Génie des Procédés

## Résolution Numérique avec GEKKO, CasADi et Julia

---

**Objectifs pédagogiques :**
- Comprendre la structure des systèmes DAE (Differential-Algebraic Equations)
- Modéliser des procédés chimiques avec équilibres thermodynamiques
- Utiliser des solveurs DAE modernes : GEKKO (Python), Julia
- Appliquer sur 3 cas industriels : Flash, CSTR, Réacteur triphasique

**Prérequis :**
- Thermodynamique des équilibres (L-V, chimique)
- Bilans de matière et d'énergie
- Python (NumPy, Matplotlib)

**Durée estimée :** 2-3 heures

---

## 📚 Introduction : ODE vs DAE

### Système ODE (Ordinary Differential Equations)

$$\frac{d\mathbf{y}}{dt} = f(t, \mathbf{y})$$

Toutes les variables ont une dérivée temporelle explicite.

### Système DAE (Differential-Algebraic Equations)

$$\begin{cases}
\frac{d\mathbf{y}}{dt} = f(t, \mathbf{y}, \mathbf{z}) & \text{(équations différentielles)} \\
0 = g(t, \mathbf{y}, \mathbf{z}) & \text{(contraintes algébriques)}
\end{cases}$$

Les variables $\mathbf{z}$ (algébriques) n'ont pas de dérivée - elles sont déterminées par les contraintes.

### Exemples en Génie des Procédés

| Procédé | Variables différentielles | Contraintes algébriques |
|---------|---------------------------|-------------------------|
| Flash drum | Holdups molaires | Équilibres L-V, sommations |
| CSTR | Concentrations, T | Équilibre chimique rapide |
| Distillation | Rétentions par plateau | MESH (équilibres L-V) |
| Réacteur catalytique | Conc. phase fluide | Efficacité catalyseur (Thiele) |

### Pourquoi SciPy ne suffit pas ?

`scipy.integrate.solve_ivp` résout des **ODE**, pas des DAE. Pour les DAE, il faut :
- **GEKKO** : Solveur DAE intégré, syntaxe simple
- **CasADi** : Interface SUNDIALS IDAS, différentiation automatique
- **Julia DifferentialEquations.jl** : Meilleur écosystème DAE

---

## 🔧 Installation des Solveurs

```bash
# GEKKO (recommandé pour ce notebook)
pip install gekko

# CasADi (optionnel)
pip install casadi

# Julia + PyJulia (pour la Partie IV)
# 1. Installer Julia : https://julialang.org/downloads/
# 2. pip install julia
# 3. python -c "import julia; julia.install()"
# 4. Dans Julia : ] add DifferentialEquations
```

In [ ]:
# Imports communs
import numpy as np
import matplotlib.pyplot as plt

# Style des graphiques
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Vérification GEKKO
try:
    from gekko import GEKKO
    print("✅ GEKKO disponible")
except ImportError:
    print("❌ GEKKO non installé : pip install gekko")

# Vérification Julia (optionnel)
try:
    from julia import Main as jl
    print("✅ PyJulia disponible")
except:
    print("⚠️ PyJulia non disponible (optionnel pour Partie IV)")

---

# PARTIE II - Flash Drum Isotherme (GEKKO)

## 🧪 Contexte

Séparation liquide-vapeur d'un mélange binaire **A (léger) / B (lourd)**.

![Flash Drum Schema](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Flash_drum.svg/300px-Flash_drum.svg.png)

### Données du Problème

- **Alimentation** : $F$ = 100 mol/s, $z_A$ = 0.5 (équimolaire)
- **Pression** : $P$ = 1 atm (constante)
- **Température** : $T$ = 350 K (isotherme)
- **Composants fictifs** A et B avec paramètres Antoine simplifiés
- **Non-idéalité** : Modèle de Margules symétrique

### Modèle Mathématique

**Variables différentielles** (dynamique du holdup) :
$$\frac{dM_L}{dt} = F - L - V$$
$$\frac{d(M_L \cdot x_A)}{dt} = F \cdot z_A - L \cdot x_A - V \cdot y_A$$

**Contraintes algébriques** (équilibres instantanés) :
$$y_A = K_A \cdot x_A \quad \text{avec} \quad K_A = \frac{\gamma_A \cdot P_A^{sat}}{P}$$
$$x_A + x_B = 1$$
$$y_A + y_B = 1$$

**Coefficients d'activité (Margules symétrique)** :
$$\ln \gamma_A = A \cdot x_B^2 \quad ; \quad \ln \gamma_B = A \cdot x_A^2$$

---

In [ ]:
# ============================================================
# PARTIE II - Flash Drum avec GEKKO
# ============================================================

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt

# --- Paramètres thermodynamiques ---

# Pression de vapeur saturante (Antoine simplifié)
# log10(Psat) = A - B/(T+C)  avec Psat en mmHg, T en °C
def Psat_A(T_K):
    """Composant A (léger, type acétone)"""
    T_C = T_K - 273.15
    return 10**(7.02 - 1200/(T_C + 230)) / 760  # atm

def Psat_B(T_K):
    """Composant B (lourd, type butanol)"""
    T_C = T_K - 273.15
    return 10**(6.50 - 1400/(T_C + 200)) / 760  # atm

# Coefficient de Margules (paramètre d'interaction)
A_marg = 1.5  # Système modérément non-idéal

def gamma_margules(x_A, A=A_marg):
    """Coefficients d'activité Margules symétrique"""
    x_B = 1 - x_A
    gamma_A = np.exp(A * x_B**2)
    gamma_B = np.exp(A * x_A**2)
    return gamma_A, gamma_B

# --- Conditions opératoires ---
T = 350  # K
P = 1.0  # atm
F = 100  # mol/s (alimentation)
z_A = 0.5  # fraction molaire A dans l'alimentation

# Vérification des Psat
print(f"=== Paramètres Flash Drum ===")
print(f"T = {T} K, P = {P} atm")
print(f"Psat_A({T} K) = {Psat_A(T):.3f} atm")
print(f"Psat_B({T} K) = {Psat_B(T):.3f} atm")
print(f"Margules A = {A_marg}")

In [ ]:
# ============================================================
# Modèle DAE Flash Drum avec GEKKO
# ============================================================

m = GEKKO(remote=False)  # Mode local

# --- Temps de simulation ---
n_points = 101
m.time = np.linspace(0, 50, n_points)  # 0 à 50 secondes

# --- Paramètres fixes ---
Psat_A_val = Psat_A(T)
Psat_B_val = Psat_B(T)

# --- Variables différentielles (états) ---
M_L = m.Var(value=500, lb=10, name='M_L')  # Holdup liquide [mol]
M_A = m.Var(value=250, lb=0, name='M_A')   # Moles de A dans liquide

# --- Variables algébriques ---
x_A = m.Var(value=0.5, lb=0.001, ub=0.999, name='x_A')  # Fraction liquide A
x_B = m.Var(value=0.5, lb=0.001, ub=0.999, name='x_B')  # Fraction liquide B
y_A = m.Var(value=0.7, lb=0.001, ub=0.999, name='y_A')  # Fraction vapeur A
y_B = m.Var(value=0.3, lb=0.001, ub=0.999, name='y_B')  # Fraction vapeur B
gamma_A = m.Var(value=1.0, lb=0.1, name='gamma_A')  # Coeff activité A
gamma_B = m.Var(value=1.0, lb=0.1, name='gamma_B')  # Coeff activité B
K_A = m.Var(value=1.5, lb=0.01, name='K_A')  # Volatilité A
K_B = m.Var(value=0.5, lb=0.01, name='K_B')  # Volatilité B
L = m.Var(value=50, lb=0, name='L')  # Débit liquide sortie [mol/s]
V = m.Var(value=50, lb=0, name='V')  # Débit vapeur sortie [mol/s]

# --- Entrée (alimentation variable) ---
# Perturbation : z_A passe de 0.5 à 0.7 à t=15s
z_A_input = m.Param(value=[0.5 if t < 15 else 0.7 for t in m.time])

# --- Équations du modèle DAE ---

# 1. Définition de x_A à partir du holdup
m.Equation(x_A * M_L == M_A)

# 2. Sommation fractions liquide
m.Equation(x_A + x_B == 1)

# 3. Coefficients d'activité Margules
m.Equation(gamma_A == m.exp(A_marg * x_B**2))
m.Equation(gamma_B == m.exp(A_marg * x_A**2))

# 4. Volatilités (Raoult modifié)
m.Equation(K_A == gamma_A * Psat_A_val / P)
m.Equation(K_B == gamma_B * Psat_B_val / P)

# 5. Équilibres liquide-vapeur
m.Equation(y_A == K_A * x_A)
m.Equation(y_B == K_B * x_B)

# 6. Sommation fractions vapeur (contrainte de normalisation)
m.Equation(y_A + y_B == 1)

# 7. Spécification : ratio L/V fixé (simplification)
m.Equation(L == 0.5 * F)  # 50% liquide, 50% vapeur
m.Equation(V == F - L)

# 8. Bilans de matière (équations différentielles)
m.Equation(M_L.dt() == F - L - V)
m.Equation(M_A.dt() == F * z_A_input - L * x_A - V * y_A)

# --- Options de résolution ---
m.options.IMODE = 4  # Simulation dynamique
m.options.SOLVER = 3  # IPOPT
m.options.NODES = 3   # Nœuds de collocation

print("✅ Modèle DAE Flash Drum construit")
print(f"   Variables : {len(m._variables)}")
print(f"   Équations : {len(m._equations)}")

In [ ]:
# ============================================================
# Résolution du système DAE
# ============================================================

print("⏳ Résolution du système DAE...")
try:
    m.solve(disp=False)
    print("✅ Résolution réussie !")
    
    # Extraction des résultats
    t = m.time
    M_L_sol = np.array(M_L.value)
    x_A_sol = np.array(x_A.value)
    y_A_sol = np.array(y_A.value)
    gamma_A_sol = np.array(gamma_A.value)
    gamma_B_sol = np.array(gamma_B.value)
    K_A_sol = np.array(K_A.value)
    
    print(f"\n📊 Résultats à t=50s (régime permanent) :")
    print(f"   x_A = {x_A_sol[-1]:.4f}")
    print(f"   y_A = {y_A_sol[-1]:.4f}")
    print(f"   γ_A = {gamma_A_sol[-1]:.4f}")
    print(f"   γ_B = {gamma_B_sol[-1]:.4f}")
    print(f"   K_A = {K_A_sol[-1]:.4f}")
    
except Exception as e:
    print(f"❌ Erreur : {e}")

In [ ]:
# ============================================================
# Visualisation des résultats
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Compositions liquide et vapeur
ax1 = axes[0, 0]
ax1.plot(t, x_A_sol, 'b-', linewidth=2, label='$x_A$ (liquide)')
ax1.plot(t, y_A_sol, 'r--', linewidth=2, label='$y_A$ (vapeur)')
ax1.axvline(x=15, color='gray', linestyle=':', label='Perturbation $z_A$')
ax1.set_xlabel('Temps [s]')
ax1.set_ylabel('Fraction molaire A')
ax1.set_title('Compositions Liquide et Vapeur', fontweight='bold')
ax1.legend()
ax1.set_ylim([0, 1])

# 2. Coefficients d'activité
ax2 = axes[0, 1]
ax2.plot(t, gamma_A_sol, 'g-', linewidth=2, label='$\\gamma_A$')
ax2.plot(t, gamma_B_sol, 'm-', linewidth=2, label='$\\gamma_B$')
ax2.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Idéal')
ax2.set_xlabel('Temps [s]')
ax2.set_ylabel('Coefficient d\'activité')
ax2.set_title('Non-idéalité (Margules)', fontweight='bold')
ax2.legend()

# 3. Holdup liquide
ax3 = axes[1, 0]
ax3.plot(t, M_L_sol, 'c-', linewidth=2)
ax3.set_xlabel('Temps [s]')
ax3.set_ylabel('Holdup liquide [mol]')
ax3.set_title('Holdup Liquide $M_L$', fontweight='bold')

# 4. Diagramme x-y
ax4 = axes[1, 1]
ax4.plot(x_A_sol, y_A_sol, 'ko-', markersize=3, label='Trajectoire')
ax4.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='y=x')
ax4.scatter(x_A_sol[0], y_A_sol[0], s=100, c='green', marker='o', label='Initial', zorder=5)
ax4.scatter(x_A_sol[-1], y_A_sol[-1], s=100, c='red', marker='s', label='Final', zorder=5)
ax4.set_xlabel('$x_A$ (liquide)')
ax4.set_ylabel('$y_A$ (vapeur)')
ax4.set_title('Diagramme x-y (Équilibre L-V)', fontweight='bold')
ax4.legend()
ax4.set_xlim([0, 1])
ax4.set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\n💡 Observation : Le système DAE capture l'équilibre L-V dynamique")
print("   avec les coefficients d'activité γ calculés à chaque instant.")

---

# PARTIE III - CSTR avec Équilibre Chimique (GEKKO)

## 🧪 Contexte

Réacteur continu parfaitement agité (CSTR) avec réaction **réversible rapide** :

$$A + B \rightleftharpoons C$$

L'équilibre chimique est atteint instantanément → **contrainte algébrique**.

### Modèle Mathématique

**Variables différentielles** :
- $C_A$ : concentration de A [mol/L]
- $C_B$ : concentration de B [mol/L]
- $T$ : température du réacteur [K]

**Variables algébriques** :
- $C_C$ : concentration de C (déterminée par l'équilibre)
- $K_{eq}$ : constante d'équilibre (fonction de T)

**Équations différentielles** :
$$\frac{dC_A}{dt} = \frac{F}{V}(C_{A,0} - C_A) - r$$
$$\frac{dC_B}{dt} = \frac{F}{V}(C_{B,0} - C_B) - r$$
$$\frac{dT}{dt} = \frac{F}{V}(T_0 - T) + \frac{(-\Delta H_r) \cdot r}{\rho C_p} - \frac{UA}{V \rho C_p}(T - T_c)$$

**Contraintes algébriques (équilibre chimique)** :
$$K_{eq}(T) = K_0 \cdot \exp\left(-\frac{E_a}{R}\left(\frac{1}{T} - \frac{1}{T_{ref}}\right)\right)$$
$$K_{eq} = \frac{C_C}{C_A \cdot C_B}$$

---

In [ ]:
# ============================================================
# PARTIE III - CSTR avec Équilibre Chimique
# ============================================================

from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt

# --- Paramètres du réacteur ---
V_reactor = 100  # Volume réacteur [L]
F_flow = 10      # Débit volumique [L/s]
tau = V_reactor / F_flow  # Temps de séjour [s]

# Concentrations d'entrée
C_A0 = 2.0  # mol/L
C_B0 = 2.5  # mol/L
T0 = 300    # K (température alimentation)

# Paramètres thermodynamiques
K0 = 10.0        # Constante d'équilibre à T_ref
T_ref = 350      # K
Ea_R = 5000      # Ea/R [K]
delta_Hr = -50000  # Enthalpie de réaction [J/mol] (exothermique)
rho_Cp = 4000    # ρ·Cp [J/(L·K)]

# Refroidissement
UA = 500     # Coefficient global [W/K]
T_cool = 290  # Température fluide refroidissement [K]

print(f"=== Paramètres CSTR ===")
print(f"Volume = {V_reactor} L, Débit = {F_flow} L/s, τ = {tau} s")
print(f"C_A0 = {C_A0} mol/L, C_B0 = {C_B0} mol/L")
print(f"K_eq(350K) = {K0}, ΔH_r = {delta_Hr/1000} kJ/mol")

In [ ]:
# ============================================================
# Modèle DAE CSTR avec GEKKO
# ============================================================

m2 = GEKKO(remote=False)

# --- Temps de simulation ---
n_pts = 201
m2.time = np.linspace(0, 100, n_pts)  # 0 à 100 s

# --- Variables différentielles (états) ---
C_A = m2.Var(value=0.5, lb=0.001, name='C_A')  # Concentration A
C_B = m2.Var(value=0.8, lb=0.001, name='C_B')  # Concentration B
T = m2.Var(value=320, lb=280, ub=450, name='T')  # Température

# --- Variables algébriques ---
C_C = m2.Var(value=0.2, lb=0, name='C_C')      # Concentration C (équilibre)
K_eq = m2.Var(value=5, lb=0.01, name='K_eq')   # Constante équilibre
r = m2.Var(value=0, name='r')                   # Vitesse nette réaction

# --- Perturbation : augmentation de C_A0 à t=30s ---
C_A0_input = m2.Param(value=[C_A0 if t < 30 else 3.0 for t in m2.time])

# --- Équations du modèle DAE ---

# 1. Constante d'équilibre (Van't Hoff)
m2.Equation(K_eq == K0 * m2.exp(-Ea_R * (1/T - 1/T_ref)))

# 2. Loi d'action de masse (contrainte algébrique)
#    K_eq = C_C / (C_A * C_B)
#    Reformulée pour éviter division par zéro
m2.Equation(C_C == K_eq * C_A * C_B)

# 3. Bilan global pour calculer r
#    À l'équilibre, on utilise un bilan matière :
#    Entrée A - Sortie A - réaction = accumulation
#    En régime pseudo-permanent sur l'équilibre : r s'ajuste
m2.Equation(r == (F_flow/V_reactor) * (C_A0_input - C_A) - C_A.dt())

# 4. Bilans de matière (équations différentielles)
m2.Equation(C_A.dt() == (F_flow/V_reactor) * (C_A0_input - C_A) - r)
m2.Equation(C_B.dt() == (F_flow/V_reactor) * (C_B0 - C_B) - r)

# 5. Bilan d'énergie
Q_react = (-delta_Hr) * r / rho_Cp  # Terme source réaction [K/s]
Q_cool = UA / (V_reactor * rho_Cp) * (T - T_cool)  # Refroidissement [K/s]
m2.Equation(T.dt() == (F_flow/V_reactor) * (T0 - T) + Q_react - Q_cool)

# --- Options ---
m2.options.IMODE = 4
m2.options.SOLVER = 3
m2.options.NODES = 3

print("✅ Modèle DAE CSTR construit")

In [ ]:
# ============================================================
# Résolution CSTR
# ============================================================

print("⏳ Résolution du système DAE CSTR...")
try:
    m2.solve(disp=False)
    print("✅ Résolution réussie !")
    
    # Extraction
    t2 = m2.time
    C_A_sol = np.array(C_A.value)
    C_B_sol = np.array(C_B.value)
    C_C_sol = np.array(C_C.value)
    T_sol = np.array(T.value)
    K_eq_sol = np.array(K_eq.value)
    
    print(f"\n📊 État final (t=100s) :")
    print(f"   C_A = {C_A_sol[-1]:.4f} mol/L")
    print(f"   C_B = {C_B_sol[-1]:.4f} mol/L")
    print(f"   C_C = {C_C_sol[-1]:.4f} mol/L")
    print(f"   T   = {T_sol[-1]:.1f} K")
    print(f"   K_eq = {K_eq_sol[-1]:.4f}")
    
    # Vérification équilibre
    K_check = C_C_sol[-1] / (C_A_sol[-1] * C_B_sol[-1])
    print(f"   Vérif: C_C/(C_A·C_B) = {K_check:.4f}")
    
except Exception as e:
    print(f"❌ Erreur : {e}")

In [ ]:
# ============================================================
# Visualisation CSTR
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Concentrations
ax1 = axes[0, 0]
ax1.plot(t2, C_A_sol, 'b-', linewidth=2, label='$C_A$')
ax1.plot(t2, C_B_sol, 'r-', linewidth=2, label='$C_B$')
ax1.plot(t2, C_C_sol, 'g-', linewidth=2, label='$C_C$ (équilibre)')
ax1.axvline(x=30, color='gray', linestyle=':', label='Perturbation')
ax1.set_xlabel('Temps [s]')
ax1.set_ylabel('Concentration [mol/L]')
ax1.set_title('Évolution des Concentrations', fontweight='bold')
ax1.legend()

# 2. Température
ax2 = axes[0, 1]
ax2.plot(t2, T_sol, 'orange', linewidth=2)
ax2.axhline(y=T_cool, color='blue', linestyle='--', alpha=0.5, label=f'$T_{{cool}}$={T_cool}K')
ax2.set_xlabel('Temps [s]')
ax2.set_ylabel('Température [K]')
ax2.set_title('Température du Réacteur', fontweight='bold')
ax2.legend()

# 3. Constante d'équilibre
ax3 = axes[1, 0]
ax3.plot(t2, K_eq_sol, 'm-', linewidth=2)
ax3.set_xlabel('Temps [s]')
ax3.set_ylabel('$K_{eq}$')
ax3.set_title('Constante d\'Équilibre (fonction de T)', fontweight='bold')

# 4. Vérification contrainte algébrique
ax4 = axes[1, 1]
ratio = C_C_sol / (C_A_sol * C_B_sol + 1e-10)
ax4.plot(t2, ratio, 'k-', linewidth=2, label='$C_C/(C_A \\cdot C_B)$')
ax4.plot(t2, K_eq_sol, 'r--', linewidth=2, label='$K_{eq}$')
ax4.set_xlabel('Temps [s]')
ax4.set_ylabel('Valeur')
ax4.set_title('Vérification Équilibre Chimique', fontweight='bold')
ax4.legend()

plt.tight_layout()
plt.show()

print("\n💡 Le DAE garantit que C_C/(C_A·C_B) = K_eq(T) à chaque instant.")
print("   C'est impossible à obtenir avec un simple ODE !")

---

# PARTIE IV - Réacteur Triphasique G-L-S (Julia)

## 🧪 Contexte

Réacteur slurry avec :
- **Phase gaz** : réactif A (ex: H₂)
- **Phase liquide** : solvant + réactif B dissous + produit C
- **Phase solide** : catalyseur en suspension

**Réaction** : $A_{(g)} + B_{(l)} \xrightarrow{cat} C_{(l)}$

### Phénomènes de Transport

1. **Transfert gaz-liquide** : A passe du gaz au liquide (loi de Henry)
2. **Diffusion dans le catalyseur** : efficacité η (module de Thiele)
3. **Réaction** : sur la surface catalytique

### Modèle DAE

**Différentielles** :
$$\frac{dC_{A,L}}{dt} = k_L a (C_A^* - C_{A,L}) - \eta \cdot k_r \cdot C_{A,L} \cdot C_B \cdot \frac{m_{cat}}{V_L}$$
$$\frac{dC_B}{dt} = \frac{F}{V}(C_{B,0} - C_B) - \eta \cdot k_r \cdot C_{A,L} \cdot C_B \cdot \frac{m_{cat}}{V_L}$$
$$\frac{dC_C}{dt} = -\frac{F}{V} C_C + \eta \cdot k_r \cdot C_{A,L} \cdot C_B \cdot \frac{m_{cat}}{V_L}$$

**Algébriques** :
$$C_A^* = H \cdot P_A \quad \text{(équilibre Henry)}$$
$$\phi = R_p \sqrt{\frac{k_r \cdot C_B}{D_{eff}}} \quad \text{(module de Thiele)}$$
$$\eta = \frac{\tanh(\phi)}{\phi} \quad \text{(efficacité catalyseur, géométrie plane)}$$

---

In [ ]:
# ============================================================
# PARTIE IV - Réacteur Triphasique avec Julia
# ============================================================

# On utilise PyJulia pour appeler Julia depuis Python
# Cela permet de garder tout dans un seul notebook

julia_code = '''
# ===========================================
# Réacteur Triphasique G-L-S en Julia
# ===========================================

using DifferentialEquations

# --- Paramètres ---
const k_L_a = 0.1      # Coefficient transfert G-L [1/s]
const H = 0.001        # Constante Henry [mol/(L·Pa)]
const P_A = 1e5        # Pression partielle A [Pa]
const k_r = 0.5        # Constante cinétique [L/(mol·s)]
const R_p = 0.001      # Rayon particule catalyseur [m]
const D_eff = 1e-9     # Diffusivité effective [m²/s]
const m_cat = 10.0     # Masse catalyseur [kg]
const V_L = 100.0      # Volume liquide [L]
const F_V = 1.0        # Débit volumique [L/s]
const C_B0 = 1.0       # Concentration B entrée [mol/L]

# Concentration A à l'équilibre G-L
const C_A_star = H * P_A  # mol/L

# --- Fonction DAE (forme résidu) ---
function reactor_gls!(resid, du, u, p, t)
    # États : u = [C_A_L, C_B, C_C, phi, eta]
    C_A_L, C_B, C_C, phi, eta = u
    dC_A_L, dC_B, dC_C, _, _ = du
    
    # Sécurités
    C_B_safe = max(C_B, 1e-10)
    phi_safe = max(phi, 1e-10)
    
    # Vitesse de réaction apparente
    r_app = eta * k_r * C_A_L * C_B_safe * (m_cat / V_L)
    
    # Résidus des équations différentielles
    resid[1] = dC_A_L - (k_L_a * (C_A_star - C_A_L) - r_app)
    resid[2] = dC_B - ((F_V/V_L) * (C_B0 - C_B) - r_app)
    resid[3] = dC_C - (-(F_V/V_L) * C_C + r_app)
    
    # Résidus des contraintes algébriques
    # Module de Thiele
    resid[4] = phi - R_p * sqrt(k_r * C_B_safe / D_eff)
    # Efficacité catalyseur
    resid[5] = eta - tanh(phi_safe) / phi_safe
    
    return nothing
end

# --- Conditions initiales ---
u0 = [0.01, 0.8, 0.0, 0.5, 0.9]  # [C_A_L, C_B, C_C, phi, eta]
du0 = [0.0, 0.0, 0.0, 0.0, 0.0]  # Dérivées initiales

# Spécification : variables différentielles vs algébriques
differential_vars = [true, true, true, false, false]

# --- Définition du problème DAE ---
tspan = (0.0, 200.0)
prob = DAEProblem(reactor_gls!, du0, u0, tspan, 
                  differential_vars=differential_vars)

# --- Résolution avec IDA (SUNDIALS) ---
sol = solve(prob, IDA(), saveat=1.0)

println("✅ Résolution Julia terminée !")
println("   $(length(sol.t)) points calculés")

# Extraction pour Python
t_julia = sol.t
C_A_L_julia = [sol.u[i][1] for i in 1:length(sol.t)]
C_B_julia = [sol.u[i][2] for i in 1:length(sol.t)]
C_C_julia = [sol.u[i][3] for i in 1:length(sol.t)]
phi_julia = [sol.u[i][4] for i in 1:length(sol.t)]
eta_julia = [sol.u[i][5] for i in 1:length(sol.t)]
'''

print("📜 Code Julia pour le réacteur triphasique :")
print("="*50)
print(julia_code[:2000] + "\n...")

In [ ]:
# ============================================================
# Exécution Julia via PyJulia
# ============================================================

try:
    from julia import Main as jl
    
    print("⏳ Exécution du code Julia...")
    jl.eval(julia_code)
    
    # Récupération des résultats en Python
    t_jl = np.array(jl.t_julia)
    C_A_L_jl = np.array(jl.C_A_L_julia)
    C_B_jl = np.array(jl.C_B_julia)
    C_C_jl = np.array(jl.C_C_julia)
    phi_jl = np.array(jl.phi_julia)
    eta_jl = np.array(jl.eta_julia)
    
    print(f"\n📊 Résultats Julia (t=200s) :")
    print(f"   C_A,L = {C_A_L_jl[-1]:.6f} mol/L")
    print(f"   C_B   = {C_B_jl[-1]:.4f} mol/L")
    print(f"   C_C   = {C_C_jl[-1]:.4f} mol/L")
    print(f"   φ     = {phi_jl[-1]:.4f}")
    print(f"   η     = {eta_jl[-1]:.4f}")
    
    JULIA_OK = True
    
except Exception as e:
    print(f"⚠️ Julia non disponible : {e}")
    print("\n📋 Pour exécuter ce code :")
    print("   1. Installer Julia : https://julialang.org")
    print("   2. pip install julia")
    print("   3. python -c 'import julia; julia.install()'")
    print("   4. Dans Julia : ] add DifferentialEquations")
    print("\n💡 Le code est affiché ci-dessus pour référence.")
    JULIA_OK = False
    
    # Génération de données fictives pour la visualisation
    t_jl = np.linspace(0, 200, 201)
    C_A_L_jl = 0.08 * (1 - np.exp(-t_jl/30))
    C_B_jl = 0.8 * np.exp(-t_jl/100) + 0.2
    C_C_jl = 0.6 * (1 - np.exp(-t_jl/50))
    phi_jl = 0.5 * np.ones_like(t_jl)
    eta_jl = 0.92 * np.ones_like(t_jl)
    print("\n🎭 Données simulées affichées pour illustration")

In [ ]:
# ============================================================
# Visualisation Réacteur Triphasique
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

title_suffix = "" if JULIA_OK else " (données simulées)"

# 1. Concentrations
ax1 = axes[0, 0]
ax1.plot(t_jl, C_A_L_jl, 'b-', linewidth=2, label='$C_{A,L}$ (A dissous)')
ax1.plot(t_jl, C_B_jl, 'r-', linewidth=2, label='$C_B$')
ax1.plot(t_jl, C_C_jl, 'g-', linewidth=2, label='$C_C$ (produit)')
ax1.set_xlabel('Temps [s]')
ax1.set_ylabel('Concentration [mol/L]')
ax1.set_title(f'Concentrations en Phase Liquide{title_suffix}', fontweight='bold')
ax1.legend()

# 2. Module de Thiele et efficacité
ax2 = axes[0, 1]
ax2.plot(t_jl, phi_jl, 'm-', linewidth=2, label='$\\phi$ (Thiele)')
ax2.set_xlabel('Temps [s]')
ax2.set_ylabel('Module de Thiele $\\phi$')
ax2.set_title('Module de Thiele', fontweight='bold')
ax2.legend()

ax2b = ax2.twinx()
ax2b.plot(t_jl, eta_jl, 'c--', linewidth=2, label='$\\eta$ (efficacité)')
ax2b.set_ylabel('Efficacité $\\eta$', color='c')
ax2b.legend(loc='lower right')

# 3. Transfert G-L
ax3 = axes[1, 0]
C_A_star = 0.001 * 1e5  # H * P_A
ax3.axhline(y=C_A_star, color='k', linestyle='--', label=f'$C_A^*$ = {C_A_star:.3f} mol/L')
ax3.plot(t_jl, C_A_L_jl, 'b-', linewidth=2, label='$C_{A,L}$')
ax3.fill_between(t_jl, C_A_L_jl, C_A_star, alpha=0.3, label='Force motrice transfert')
ax3.set_xlabel('Temps [s]')
ax3.set_ylabel('Concentration A [mol/L]')
ax3.set_title('Transfert Gaz-Liquide', fontweight='bold')
ax3.legend()

# 4. Conversion
ax4 = axes[1, 1]
C_B0_val = 0.8
conversion = 100 * (C_B0_val - C_B_jl) / C_B0_val
ax4.plot(t_jl, conversion, 'orange', linewidth=2)
ax4.set_xlabel('Temps [s]')
ax4.set_ylabel('Conversion B [%]')
ax4.set_title('Conversion du Réactif B', fontweight='bold')
ax4.set_ylim([0, 100])

plt.tight_layout()
plt.show()

print("\n💡 Ce système DAE couple :")
print("   - Transfert G-L (équilibre Henry)")
print("   - Diffusion intraparticulaire (module de Thiele)")
print("   - Réaction catalytique hétérogène")

---

# 📚 Synthèse et Comparaison des Solveurs

## Tableau Récapitulatif

| Critère | GEKKO | CasADi | Julia DiffEq |
|---------|-------|--------|---------------|
| **Installation** | `pip install gekko` | `pip install casadi` | Julia + packages |
| **Syntaxe** | Intuitive (équations) | Symbolique | Proche MATLAB |
| **Backend DAE** | APMonitor/IPOPT | SUNDIALS IDAS | SUNDIALS IDA |
| **Performance** | Bonne | Très bonne | Excellente |
| **Cas d'usage** | Procédés, contrôle | Optimisation, MPC | Recherche, HPC |
| **Courbe apprentissage** | Faible | Moyenne | Moyenne |

## Recommandations

1. **Débutant / Prototypage** → **GEKKO** : syntaxe simple, tout intégré
2. **Optimisation dynamique / MPC** → **CasADi** : différentiation auto, IPOPT
3. **Performance / Grande échelle** → **Julia** : compilé, parallélisable

## Ce que vous avez appris

✅ Structure des systèmes DAE (différentielles + algébriques)  
✅ Modélisation d'équilibres thermodynamiques (L-V, chimique)  
✅ Coefficients d'activité (Margules)  
✅ Transfert de matière G-L (Henry)  
✅ Efficacité catalytique (Thiele)  
✅ Utilisation de GEKKO et Julia pour les DAE

---

## 📖 Références

- [GEKKO Documentation](https://gekko.readthedocs.io/)
- [CasADi](https://web.casadi.org/)
- [DifferentialEquations.jl](https://diffeq.sciml.ai/)
- Fogler, H.S. - *Elements of Chemical Reaction Engineering*
- Smith, Van Ness, Abbott - *Introduction to Chemical Engineering Thermodynamics*